In [18]:
from datetime import datetime
import pandas as pd
import requests

# Current utc date and time
utcnow = datetime.utcnow().strftime("%Y%m%d_%H%M")
dateoftoday = datetime.utcnow().strftime("%Y%m%d")

# Varible to keep the count of the listings.    
listings_count = 0

area_info={
    "gamla_stan": "ab85609a4dbfa32c62a9a8b126550ced2d074ff5",
    "gardet": "39992548ad9d0cd15497e4e3b3d1519f0e91f5aa",
    "hjorthagen": "ad226147ded7e9ee32d5701aae0658bb91166d0c",
    "kungsholmen": "a92006fdcdf6dd843da1802dd5b5ba13e57d26df",
    "lilla_essingen": "af901ec433dffd143a54fb1a2c24a5aa7ad46089",
    "norra_djurgarden": "91d4716f82833881428eaf46da90871b616396b3",
    "norrmalm": "b65b348d26772bbfcdab88057996e136291bfb9c",
    "ostermalm": "fedafc35d25c3c294925905e0bd1547f192fde7b",
    "sodermalm": "46df34129aa827f7b5862622fa23b574ae5e5ba6",
    "solna": "b88ff1d7604ab8737572a032f39c270ef013c123",
    "stora_essingen": "fd879c94aa92827c527a7791cca091e192f9d49e",
    "sundbyberg": "44265ea0b97556e9e3a3ef9118482db8a7fab2b6",
    "vasastan": "72f539037b60f5ff2d6a9133f78e9ea8983f2ada"
}

print(type(area_info))
 

<class 'dict'>


In [19]:
area_info.items()

dict_items([('gamla_stan', 'ab85609a4dbfa32c62a9a8b126550ced2d074ff5'), ('gardet', '39992548ad9d0cd15497e4e3b3d1519f0e91f5aa'), ('hjorthagen', 'ad226147ded7e9ee32d5701aae0658bb91166d0c'), ('kungsholmen', 'a92006fdcdf6dd843da1802dd5b5ba13e57d26df'), ('lilla_essingen', 'af901ec433dffd143a54fb1a2c24a5aa7ad46089'), ('norra_djurgarden', '91d4716f82833881428eaf46da90871b616396b3'), ('norrmalm', 'b65b348d26772bbfcdab88057996e136291bfb9c'), ('ostermalm', 'fedafc35d25c3c294925905e0bd1547f192fde7b'), ('sodermalm', '46df34129aa827f7b5862622fa23b574ae5e5ba6'), ('solna', 'b88ff1d7604ab8737572a032f39c270ef013c123'), ('stora_essingen', 'fd879c94aa92827c527a7791cca091e192f9d49e'), ('sundbyberg', '44265ea0b97556e9e3a3ef9118482db8a7fab2b6'), ('vasastan', '72f539037b60f5ff2d6a9133f78e9ea8983f2ada')])

In [30]:
# Loop thruogh all the areas one by one
for area,area_code in area_info.items():
    payload={}
    headers = {"User-Agent": "Mozilla/5.0"
    }

    url = "https://www.hemnet.se/bostader/search/"+area_code

    response = requests.request(
        "GET",
        url,
        headers=headers,
    )

    # Store all the listings in that area as a dataframe
    df = pd.json_normalize(response.json(),
        record_path="properties",
        max_level=1
    )
    print("{:<20} {:<15}".format(area, str(df.shape)))

    # Enrich the df with info about internal area naming and date  
    df["area"] = area
    df["date"] = dateoftoday

    # Update the listings_count
    listings_count += df.shape[0]
    
    # # Save the dataframe inside directory
    # df.to_parquet(f"{directory_path}{area}.parquet.gzip", compression="gzip")
    #print(df)

print(f"\n{listings_count} listings in the areas of interest today!")

gamla_stan                (7, 39)        
gardet                    (88, 42)       
hjorthagen                (23, 39)       
kungsholmen               (233, 42)      
lilla_essingen            (15, 39)       
norra_djurgarden          (7, 39)        
norrmalm                  (24, 39)       
ostermalm                 (128, 42)      
sodermalm                 (258, 42)      
solna                     (301, 42)      
stora_essingen            (15, 39)       
sundbyberg                (160, 42)      
vasastan                  (265, 42)      

10668 listings in the areas of interest today!


In [34]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    bucket_name = "hemnet_test"
    

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )
testfilepath=r"G:\My Drive\Privat\Documents\Python\hemnet\test_hemnet\data\areas_of_interest\20221109_1814\norrmalm.parquet.gzip"
# The path to your file to upload
source_file_name = r"G:\My Drive\Privat\Documents\Python\hemnet\test_hemnet\data\areas_of_interest\20221109_1814"
# The ID of your GCS object
destination_blob_name = "norrmalm.parquet.gzip"
upload_blob("hemnet_test",testfilepath,destination_blob_name)


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started